<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/Machine-Learning/blob/main/ML/classes/class_march_3/class_march_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Exercises

Los siguientes ejercicios se basan en el conjunto de datos de vivienda de este capítulo:

1. Pruebe un regresor de máquina de soporte vectorial (sklearn.svm.SVR) con varios hiperparámetros, como `kernel="linear"` (con varios valores para el hiperparámetro C) o `kernel="rbf"` (con varios valores para los hiperparámetros C y gamma). No se preocupe por el significado de estos hiperparámetros por ahora. ¿Cómo funciona el mejor predictor de RVS?

2. Intente reemplazar GridSearchCV con RandomizedSearchCV.

3. Intente agregar un transformador en la canalización de preparación para seleccionar solo los atributos más importantes.

4. Intente crear una canalización única que haga la preparación completa de los datos más la predicción final.

5. Explore automáticamente algunas opciones de preparación utilizando GridSearchCV.

Las soluciones a estos ejercicios se pueden encontrar en los cuadernos Jupyter disponibles en
https://github.com/ageron/handson-ml2.

In [2]:
import os 
# Download the California image
images_path = os.path.join(PROJECT_ROOT_DIR, "images", "end_to_end_project")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
filename = "california.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/end_to_end_project/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

NameError: name 'PROJECT_ROOT_DIR' is not defined

# Exercise solutions

1.  **Pregunta:** pruebe con un regresor de máquina soporte vectorial (sklearn.svm.SVR), con varios hiperparámetros como kernel="linear" (con varios valores para el hiperparámetro C) o kernel="rbf" (con varios valores para C y gamma hiperparámetros). No se preocupe por el significado de estos hiperparámetros por ahora. ¿Cómo funciona el mejor predictor de RVS?

Advertencia: la siguiente celda puede tardar cerca de 30 minutos en ejecutarse, o más, dependiendo de su hardware.

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split 
from sklearn.base import BaseEstimator, TransformerMixin  

from sklearn.impute import SimpleImputer 
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OneHotEncoder 

from sklearn.svm import SVR 


In [2]:
class AdAtribComb(BaseEstimator, TransformerMixin):
    def __init__(self, ad_dph = True, ad_hph = True, ad_pph = True): # no *args or **kargs
        self.ad_dph = ad_dph
        self.ad_hph = ad_hph
        self.ad_pph = ad_pph
        
    def fit(self, X, y=None):
        return self # nothing else to do
    
    def transform(self, X, y=None):
        habitaciones, dormitorios, población, hogares = 3, 4, 5, 6 
        if self.ad_dph:
            dormitorios_por_habitación=X[:,dormitorios]/X[:,habitaciones]
            X=np.c_[X, dormitorios_por_habitación]
        if self.ad_hph:
            habitaciones_por_hogar=X[:,habitaciones]/X[:,hogares]
            X=np.c_[X, habitaciones_por_hogar]
        if self.ad_pph: 
            población_por_hogar = X[:, población]/X[:, hogares]
            X=np.c_[X, población_por_hogar] 
        return X

In [3]:
v = pd.read_csv('vivienda.csv') 
v_train,v_test = train_test_split(v,test_size = 0.2, random_state = 513) 

v = v_train.drop('precio', axis = 1)
v_labels = v_train[['precio']] 

In [4]:
v_num = v.drop('proximidad', axis = 1)
pipeline_num = Pipeline([
   ('imputador', SimpleImputer(strategy = 'median') ),
    ('adicionador_atributos',AdAtribComb(ad_dph = True, ad_hph = True, ad_pph = True)),
    ('escalador_std', StandardScaler() )
])

In [5]:
lista_atributos_num = list(v_num.columns)
lista_atributos_cat = ['proximidad']
pipeline_total = ColumnTransformer([
    ('num', pipeline_num, lista_atributos_num),
    ('cat', OneHotEncoder(), lista_atributos_cat)
])

In [6]:
x_prep = pipeline_total.fit_transform(v)

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
        {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', \
                           verbose=2)
grid_search.fit(x_prep, v_labels.values.ravel()) 

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END ..............................C=10.0, kernel=linear; total time=  17.8s
[CV] END ..............................C=10.0, kernel=linear; total time=  17.8s
[CV] END ..............................C=10.0, kernel=linear; total time=  17.8s
[CV] END ..............................C=10.0, kernel=linear; total time=  17.8s
[CV] END ..............................C=10.0, kernel=linear; total time=  17.3s
[CV] END ..............................C=30.0, kernel=linear; total time=  17.5s
[CV] END ..............................C=30.0, kernel=linear; total time=  17.9s
[CV] END ..............................C=30.0, kernel=linear; total time=  17.9s
[CV] END ..............................C=30.0, kernel=linear; total time=  18.9s
[CV] END ..............................C=30.0, kernel=linear; total time=  18.4s
[CV] END .............................C=100.0, kernel=linear; total time=  17.3s
[CV] END .............................C=100.0, 

[CV] END .....................C=10.0, gamma=0.01, kernel=rbf; total time=  31.3s
[CV] END .....................C=10.0, gamma=0.01, kernel=rbf; total time=  35.1s
[CV] END .....................C=10.0, gamma=0.01, kernel=rbf; total time=  43.2s
[CV] END .....................C=10.0, gamma=0.01, kernel=rbf; total time=  41.5s
[CV] END .....................C=10.0, gamma=0.03, kernel=rbf; total time=  34.7s
[CV] END .....................C=10.0, gamma=0.03, kernel=rbf; total time=  36.3s
[CV] END .....................C=10.0, gamma=0.03, kernel=rbf; total time=  40.7s
[CV] END .....................C=10.0, gamma=0.03, kernel=rbf; total time=  35.1s
[CV] END .....................C=10.0, gamma=0.03, kernel=rbf; total time=  34.1s
[CV] END ......................C=10.0, gamma=0.1, kernel=rbf; total time=  33.9s
[CV] END ......................C=10.0, gamma=0.1, kernel=rbf; total time=  32.4s
[CV] END ......................C=10.0, gamma=0.1, kernel=rbf; total time=  32.9s
[CV] END ...................

[CV] END .....................C=300.0, gamma=0.1, kernel=rbf; total time=  18.7s
[CV] END .....................C=300.0, gamma=0.1, kernel=rbf; total time=  18.2s
[CV] END .....................C=300.0, gamma=0.3, kernel=rbf; total time=  19.0s
[CV] END .....................C=300.0, gamma=0.3, kernel=rbf; total time=  16.9s
[CV] END .....................C=300.0, gamma=0.3, kernel=rbf; total time=  16.9s
[CV] END .....................C=300.0, gamma=0.3, kernel=rbf; total time=  17.6s
[CV] END .....................C=300.0, gamma=0.3, kernel=rbf; total time=  18.9s
[CV] END .....................C=300.0, gamma=1.0, kernel=rbf; total time=  17.5s
[CV] END .....................C=300.0, gamma=1.0, kernel=rbf; total time=  20.5s
[CV] END .....................C=300.0, gamma=1.0, kernel=rbf; total time=  18.3s
[CV] END .....................C=300.0, gamma=1.0, kernel=rbf; total time=  19.4s
[CV] END .....................C=300.0, gamma=1.0, kernel=rbf; total time=  21.3s
[CV] END ...................

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0,
                                10000.0, 30000.0],
                          'kernel': ['linear']},
                         {'C': [1.0, 3.0, 10.0, 30.0, 100.0, 300.0, 1000.0],
                          'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'kernel': ['rbf']}],
             scoring='neg_mean_squared_error', verbose=2)

El mejor modelo logra la siguiente puntuación (evaluada utilizando una validación cruzada de 5 veces):

In [8]:
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

71027.0796073586

Eso es mucho peor que `RandomForestRegressor`. Veamos los mejores hiperparámetros encontrados:

In [9]:
grid_search.best_params_

{'C': 1000.0, 'gamma': 0.1, 'kernel': 'rbf'}

El kernel lineal parece mejor que el kernel RBF. Observe que el valor de C es el valor máximo probado. Cuando esto sucede, definitivamente desea iniciar la búsqueda de cuadrícula nuevamente con valores más altos para C (eliminando los valores más pequeños), porque es probable que los valores más altos de C sean mejores.

## Punto 2 Solución

## Referencias  

* sklearn.svm.SVR: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html